In [2]:
# Core
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.special import expit as sigmoid, logit

# Symbolic math
import sympy
from sympy import diff

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Utility
from tqdm.auto import tqdm

# MSE imports
import kepler_sieve
from asteroid_data import load_ast_elt
from candidate_element import asteroid_elts, perturb_elts, random_elts
from candidate_element import score_by_elt, score_batches, report_v, plot_v
from candidate_element import mixture_log_like, mixture_objective_all, mixture_objective, mixture_jacobian, mixture_hessian
from candidate_element import hl2x, x2hl, mixture_fit
from ztf_ast import load_ztf_nearest_ast, calc_hit_freq
from ztf_element import load_ztf_batch, make_ztf_batch, ztf_elt_hash
from astro_utils import deg2dist, dist2deg

In [3]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

## Batches of Orbital Elements: Unperturbed, Perturbed, Random

In [4]:
# Load all the asteroid elements
ast_elt = load_ast_elt()

In [5]:
ast_elt

,Num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255499,1255499,2019 QG,58600.0,0.822197,0.237862,0.220677,5.066979,3.770460,0.503214,21.55,0.15,JPL 1,0.807024,272.309165,0.023074,-3.225717,-2.921908,-3.728932,-21.808984
1255501,1255501,2019 QL,58600.0,2.722045,0.530676,0.113833,4.741919,2.351059,5.297173,19.21,0.15,JPL 1,-2.082964,1640.368337,0.003830,-0.176219,-1.273172,0.809793,257.420824
1255502,1255502,2019 QQ,58600.0,1.053137,0.389091,0.172121,5.648270,2.028352,3.266522,25.31,0.15,JPL 1,-3.081905,394.753268,0.015917,-1.623227,-1.688469,1.393436,189.527723


In [6]:
# Load ztf nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [7]:
ztf_ast

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
0,b'ZTF18acebhfp',676397301515010013,14490,58430.397303,41.357345,58.879488,0.387942,0.653853,0.649598,18.946699,0.865682,1208789,0.005029,41.396388,58.592038,0.390926,0.655662,0.645976
1,b'ZTF18abodmwk',596403415715010014,5831,58350.403414,30.969721,65.305308,0.358224,0.558644,0.748059,19.010401,0.855504,1227812,0.024428,33.729101,64.536183,0.357570,0.578168,0.733393
2,b'ZTF18abodmwk',626428345715010011,10614,58380.428345,30.969705,65.305294,0.358224,0.558644,0.748059,18.935900,0.855504,1169677,0.015510,29.207596,64.817653,0.371401,0.550472,0.747691
3,b'ZTF18abodmwk',630507595715015045,11250,58384.507593,30.969940,65.305305,0.358223,0.558645,0.748059,19.260401,0.855504,1251951,0.012386,30.227911,65.945543,0.352183,0.551505,0.756181
4,b'ZTF18abodmwk',618384965715010022,9040,58372.384965,30.969643,65.305179,0.358226,0.558644,0.748058,19.220200,0.855504,1246591,0.025343,34.169666,64.771024,0.352659,0.579473,0.734740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5697957,b'ZTF20aareruw',1151532523515015015,97109,58905.532523,253.007910,55.485537,-0.165587,-0.169403,0.971537,19.192400,0.608023,1102168,0.036944,253.707834,53.408139,-0.167230,-0.205583,0.964246
5697958,b'ZTF20aarerwx',1151533002615015009,97110,58905.533009,232.886408,53.509617,-0.358833,-0.115301,0.926253,19.687099,0.559474,1028157,0.084402,224.967815,54.919912,-0.406617,-0.047125,0.912382
5697959,b'ZTF20aarerww',1151533002115010003,97110,58905.533009,236.167899,54.618457,-0.322375,-0.116973,0.939357,19.957001,0.392662,539940,0.052254,240.693936,56.155104,-0.272611,-0.115218,0.955200
5697960,b'ZTF20aarervr',1151526063515015015,97098,58905.526065,286.235286,33.876902,0.232120,-0.509626,0.828494,19.049299,0.517241,1246304,0.014054,285.998189,34.657915,0.226704,-0.499251,0.836274


In [8]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top batch_size
idx = np.argsort(hit_count)[::-1]

# Extract the asteroid number and hit count for this batch
ast_num_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [9]:
# Set batch size
# batch_size = 1000
batch_size = 64

# Batch of unperturbed elements
elts = asteroid_elts(ast_nums=ast_num_best[0:batch_size])

# Add hit_count column
elts['hit_count'] = hit_count_best[0:batch_size]

elts

,element_id,a,e,inc,Omega,omega,f,epoch,h,R,hit_count
0,51921,2.669306,0.217361,0.499554,4.699703,2.450796,-1.133491,58600.0,0.015625,0.017453,194
1,59244,2.634727,0.262503,0.465045,5.738298,1.766995,-1.601363,58600.0,0.015625,0.017453,190
2,15786,1.883227,0.047655,0.392360,6.134689,0.804823,-1.246069,58600.0,0.015625,0.017453,189
3,3904,2.556387,0.098279,0.261542,5.450163,2.202423,-1.357345,58600.0,0.015625,0.017453,188
4,142999,2.619944,0.191376,0.514017,0.238022,0.946463,-1.299301,58600.0,0.015625,0.017453,186
...,...,...,...,...,...,...,...,...,...,...,...
59,11952,2.219650,0.086091,0.117967,0.042442,2.904823,-3.016580,58600.0,0.015625,0.017453,148
60,134815,2.612770,0.140831,0.513922,0.272689,0.645552,-0.957836,58600.0,0.015625,0.017453,148
61,27860,2.619406,0.096185,0.200633,5.541400,3.266046,3.948770,58600.0,0.015625,0.017453,148
62,85937,2.342291,0.197267,0.439063,5.279693,3.210025,3.947687,58600.0,0.015625,0.017453,148


In [10]:
np.mean(elts.hit_count)

161.453125

In [11]:
# Alias elts to elts_ast for naming consistency with elts_pert and elts_rand
elts_ast = elts.copy()

In [12]:
# Perturb orbital elements
sigma_a = 0.0 
sigma_e = 0.0 
sigma_f_deg = 0.1
sigma_Omega_deg = 0.0
sigma_omega_deg = 0.0
mask_pert = None
random_seed = 42

elts_pert = perturb_elts(elts, sigma_a=sigma_a, sigma_e=sigma_e, sigma_f_deg=sigma_f_deg, 
                         sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                         mask_pert=mask_pert, random_seed=random_seed)

In [13]:
# Batch of random elements
elts_rand = random_elts(element_id_start=0, size=batch_size, random_seed=42)

## Batches of ZTF Data vs. Elements

In [14]:
# Arguments to make_ztf_batch
# thresh_deg = 10.0
thresh_deg = 1.0
near_ast = False

In [15]:
# Load unperturbed element batch
ztf_elt_ast = load_ztf_batch(elts=elts_ast, thresh_deg=thresh_deg, near_ast=near_ast)

In [16]:
# Review 
ztf_elt_ast

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
0,53851,733,b'ZTF18abnothj',594197584815010004,5501,58348.197581,266.229165,-13.513802,-0.063945,-0.983101,...,0.004080,-0.057300,-0.982042,0.179751,2.234078,0.010624,2191.408734,0.999944,0.370552,False
1,73604,733,b'ZTF18ablwzmb',594197584815015003,5501,58348.197581,265.761024,-13.509148,-0.071871,-0.982578,...,0.004080,-0.057300,-0.982042,0.179751,2.234078,0.016809,3467.151428,0.999859,0.927559,False
2,82343,733,b'ZTF18abiydvm',635193253015015018,12089,58389.193252,270.331454,-11.244934,0.005674,-0.977422,...,0.003825,0.000919,-0.977996,0.208622,2.703478,0.005450,1124.103915,0.999985,0.097503,False
3,257221,733,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,...,-0.001953,0.639004,0.610779,0.467571,2.175851,0.008713,1797.091521,0.999962,0.249197,False
4,327000,733,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,...,-0.002129,0.606278,0.637608,0.475272,2.114866,0.007949,1639.539679,0.999968,0.207419,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90206,5650588,324582,b'ZTF20aaqvhld',1150176701515015008,96618,58904.176701,44.164238,29.650540,0.623416,0.752309,...,-0.001541,0.627640,0.750696,0.206212,2.981799,0.008187,1688.638104,0.999966,0.220027,False
90207,5650589,324582,b'ZTF20aaqvhld',1150176245715015005,96617,58904.176250,44.164062,29.650536,0.623417,0.752307,...,-0.001541,0.627641,0.750695,0.206213,2.981793,0.008187,1688.601889,0.999966,0.220018,False
90208,5650665,324582,b'ZTF20aaqvhll',1150176245815015010,96617,58904.176250,44.368640,28.490480,0.628284,0.753618,...,-0.001541,0.627641,0.750695,0.206213,2.981793,0.013370,2757.856412,0.999911,0.586871,False
90209,5650697,324582,b'ZTF20aaqvhmb',1150176246015015005,96617,58904.176250,43.296207,29.505908,0.633424,0.743491,...,-0.001541,0.627641,0.750695,0.206213,2.981793,0.012388,2555.278205,0.999923,0.503822,False


In [17]:
np.sum(ztf_elt_ast.is_hit)

10333

In [18]:
# Load perturbed element batch
ztf_elt_pert = load_ztf_batch(elts=elts_pert, thresh_deg=thresh_deg, near_ast=near_ast)

In [19]:
# Review 
ztf_elt_pert

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
0,53851,733,b'ZTF18abnothj',594197584815010004,5501,58348.197581,266.229165,-13.513802,-0.063945,-0.983101,...,0.004080,-0.057059,-0.982027,0.179911,2.233772,0.010900,2248.389426,0.999941,0.390072,False
1,73604,733,b'ZTF18ablwzmb',594197584815015003,5501,58348.197581,265.761024,-13.509148,-0.071871,-0.982578,...,0.004080,-0.057059,-0.982027,0.179911,2.233772,0.017098,3526.769930,0.999854,0.959731,False
2,82343,733,b'ZTF18abiydvm',635193253015015018,12089,58389.193252,270.331454,-11.244934,0.005674,-0.977422,...,0.003825,0.001112,-0.977967,0.208755,2.703109,0.005214,1075.539469,0.999986,0.089260,False
3,257221,733,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,...,-0.001955,0.638795,0.611100,0.467437,2.176167,0.009118,1880.776044,0.999958,0.272946,False
4,327000,733,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,...,-0.002131,0.606054,0.637930,0.475125,2.115188,0.007997,1649.455847,0.999968,0.209935,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90404,5650665,324582,b'ZTF20aaqvhll',1150176245815015010,96617,58904.176250,44.368640,28.490480,0.628284,0.753618,...,-0.001539,0.628270,0.750149,0.206283,2.982643,0.013552,2795.405621,0.999908,0.602961,False
90405,5650697,324582,b'ZTF20aaqvhmb',1150176246015015005,96617,58904.176250,43.296207,29.505908,0.633424,0.743491,...,-0.001539,0.628270,0.750149,0.206283,2.982643,0.011742,2422.043031,0.999931,0.452652,False
90406,5650705,324582,b'ZTF20aaqvhlg',1150176245715015008,96617,58904.176250,44.621045,29.303550,0.620689,0.756675,...,-0.001539,0.628270,0.750149,0.206283,2.982643,0.010041,2071.206763,0.999950,0.331016,False
90407,5650759,324582,b'ZTF20aaqvhmn',1150176701415015008,96618,58904.176701,43.253607,29.953901,0.631043,0.743321,...,-0.001539,0.628269,0.750151,0.206282,2.982649,0.017307,3569.912521,0.999850,0.983355,False


In [20]:
# Load random element batch
ztf_elt_rand = load_ztf_batch(elts=elts_rand, thresh_deg=thresh_deg, near_ast=near_ast)

In [21]:
ztf_elt_rand

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
0,96823,0,b'ZTF18abvfbei',708277241115015000,18391,58462.277245,15.633115,16.050475,0.925468,0.347581,...,-0.000421,0.923639,0.350978,0.153963,2.107494,0.005081,1048.005013,0.999987,0.084748,False
1,96824,0,b'ZTF18abvfbei',720160041115010000,19954,58474.160046,15.633036,16.050428,0.925468,0.347580,...,-0.000475,0.923696,0.355835,0.142011,2.247321,0.012084,2492.496946,0.999927,0.479369,False
2,96825,0,b'ZTF18abvfbei',703201851115015000,18190,58457.201852,15.633098,16.050466,0.925468,0.347581,...,-0.000398,0.921753,0.353649,0.159071,2.052863,0.011020,2272.970994,0.999939,0.398648,False
3,96827,0,b'ZTF18abvfbei',717122091115010000,19362,58471.122095,15.633029,16.050405,0.925468,0.347579,...,-0.000461,0.924249,0.353170,0.145033,2.210267,0.008023,1654.769593,0.999968,0.211290,False
4,96828,0,b'ZTF18abvfbei',717162841115010001,19411,58471.162847,15.633006,16.050428,0.925468,0.347579,...,-0.000461,0.924245,0.353196,0.144993,2.210760,0.008069,1664.388261,0.999967,0.213753,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67956,5650542,63,b'ZTF20aaqvhhw',1150174895215015003,96614,58904.174896,55.836422,8.024680,0.556059,0.807255,...,0.000542,0.549457,0.811172,-0.200245,2.640927,0.008046,1659.654697,0.999968,0.212539,False
67957,5651189,63,b'ZTF20aaqvkuw',1150186701015015008,96637,58904.186701,55.877823,8.132705,0.555318,0.808165,...,0.000542,0.549412,0.811205,-0.200231,2.641083,0.007774,1603.459986,0.999970,0.198390,False
67958,5651330,63,b'ZTF20aaqvlet',1150186701115015002,96637,58904.186701,57.021720,8.583610,0.538224,0.820403,...,0.000542,0.549412,0.811205,-0.200231,2.641083,0.016182,3337.905000,0.999869,0.859695,False
67959,5651501,63,b'ZTF20aaqvkti',1150186700915015010,96637,58904.186701,56.117308,8.941580,0.550719,0.814245,...,0.000542,0.549412,0.811205,-0.200231,2.641083,0.016945,3495.178854,0.999856,0.942615,False


## Group Results by Element

In [22]:
# Score the three batches by element
elt_score_ast = score_by_elt(ztf_elt_ast, thresh_deg=thresh_deg, fit_mixture=True)
elt_score_pert = score_by_elt(ztf_elt_pert, thresh_deg=thresh_deg, fit_mixture=True)
elt_score_rand = score_by_elt(ztf_elt_rand, thresh_deg=thresh_deg, fit_mixture=True)

/home/michael/anaconda3/envs/kepler/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


/home/michael/IACS/kepler-sieve/src/astro_utils.py:103: RuntimeWarning: invalid value encountered in arcsin
  x_rad = np.arcsin(0.5 * dist) * 2.0


In [23]:
elt_score_ast

,n_obs,log_v_mean,log_v_std,t,hits,log_like,h,lambda,mu_sec
element_id,,,,,,,,,
733,1424,-3.143719,5.457957,118.631134,194.0,2382.783606,0.136235,1.090048e+07,11.672215
1476,936,-4.068411,6.217750,124.469453,190.0,2403.455420,0.203016,1.017194e+07,12.082982
1803,1117,-3.643090,5.723263,121.757709,189.0,2328.563326,0.169205,8.951016e+06,12.880713
2015,1087,-3.746540,6.209130,123.522233,188.0,2493.809768,0.216948,2.203928e+07,8.208757
2294,1035,-3.770222,6.126519,121.293367,186.0,2350.092598,0.179710,1.147332e+07,11.377098
...,...,...,...,...,...,...,...,...,...
203722,3127,-1.868939,3.619760,104.510291,148.0,53.838890,0.364055,2.018366e+00,27144.979399
253246,943,-3.442433,5.691039,105.711295,148.0,1827.334100,0.176478,1.302093e+07,10.679601
306781,1726,-2.437276,4.677453,101.257018,148.0,1883.298454,0.085748,2.770589e+07,7.321331


In [ ]:
elt_score_pert

In [ ]:
elt_score_rand

In [ ]:
# Make elt_score table for asteroids, perturbed, and unperturbed elements
elt_score_tbl = {
    'ast': elt_score_ast,
    'pert': elt_score_pert,
    'rand': elt_score_rand,
}

In [ ]:
# Score the three batches
batch_score = score_batches(elt_score_tbl)

In [ ]:
batch_score

## Plot Distribution of v

In [ ]:
# Report unperturbed elements
report_v(ztf_elt_ast, elt_name='unperturbed asteroid')

In [ ]:
# Report unperturbed elements
report_v(ztf_elt_pert, elt_name='perturbed asteroid')

In [ ]:
# Report random elements
report_v(ztf_elt_rand, elt_name='random')

In [ ]:
plot_v(ztf_elt_ast, 'Unperturbed Asteroid', v_max=1.0, is_cum=False)

In [ ]:
plot_v(ztf_elt_ast, 'Unperturbed Asteroid', v_max=1.0, is_cum=True)

In [ ]:
plot_v(ztf_elt_pert, 'Perturbed Asteroid', is_cum=False)

In [ ]:
plot_v(ztf_elt_rand, 'Random Elements')

## Mixture Model Fit

In [ ]:
# Extract v for first asteroid
element_id = elts.element_id[0]
mask = ztf_elt_ast.element_id == element_id
v = ztf_elt_ast[mask].v.values

# Actual hit count
# hits_actual = np.sum(ztf_elt_ast[mask].is_hit)
hits_actual = elt_score_ast.loc[element_id].hits

In [ ]:
# Test values
h = 0.01
lam = 0.5

In [ ]:
# Initial guess
h_ = logit(h)
lam_ = np.log(lam)
x0 = np.array([h_, lam_])

In [ ]:
# Evaluate log likelihood function and derivatives
L, grad_L, hess_L = mixture_log_like(h=h, lam=lam, x=v)
# Unpack gradient
dL_dh, dL_dl = grad_L

# Report results
print(f'Log Like L(h, lam) = {L:8.6f}')
print(f'dL/dh   = {dL_dh:8.6f}')
print(f'dL/dlam = {dL_dl:8.6f}')

In [ ]:
# Evaluate objective function and derivatives
F, grad_F, hess_F = mixture_objective_all(x=x0, v=v)
# Unpack gradient
dF_dx, dF_dy = grad_F

# Report results
print(f'Objective  F(x0, v) = {F:8.6f}')
print(f'dF/dx = {dF_dx:8.6f}')
print(f'dF/dy = {dF_dy:8.6f}')

## Check First Derivatives Numerically

In [ ]:
# Alias mixture_log_like to log_like and mixture_objective to objective for legibility
log_like = mixture_log_like
objective = mixture_objective
jacobian = mixture_jacobian
hessian = mixture_hessian

In [ ]:
# dL_dh by symmetric differences; L is log likelihood
dh = h * 1.0E-4
h_up = h + dh / 2
h_dn = h - dh / 2
dL_dh_num = (log_like(h_up, lam, v)[0] - log_like(h_dn, lam, v)[0]) / dh

# dF_dx by symmetric differences; F is optimization function
dx = h_ * 1.0E-4
x_dn = np.array([h_ - dx/2, lam_])
x_up = np.array([h_ + dx/2, lam_])
dF_dx_num = (objective(x_up, v) - objective(x_dn, v)) / dx

# Compare calculated and numerical derivatives for L and F
err_dL_dh = np.abs(dL_dh - dL_dh_num)
err_dF_dx = np.abs(dF_dx - dF_dx_num)

# Report results
print(f'Log Likelihood function L w.r.t h')
print(f'dL_dh_num = {dL_dh_num:10.6f}')
print(f'Error:      {err_dL_dh:10.2e}')

print(f'\nObjective function F w.r.t x')
print(f'dF_dx_num = {dF_dx_num:10.6f}')
print(f'Error:      {err_dF_dx:10.2e}')

In [ ]:
# dL_dl by symmetric differences
dlam = lam * 1.0E-4
lam_up = lam + dlam / 2
lam_dn = lam - dlam / 2
dL_dl_num = (log_like(h, lam_up, v)[0] - log_like(h, lam_dn, v)[0]) / dlam

# dF_dy by symmetric differences
dy = lam_ * 1.0E-4
y_dn = np.array([h_, lam_ - dy/2])
y_up = np.array([h_, lam_ + dy/2])
dF_dy_num = (objective(y_up, v) - objective(y_dn, v)) / dy

# Compare calculated and numerical derivatives for L and F
err_dL_dl = np.abs(dL_dl - dL_dl_num)
err_dF_dy = np.abs(dF_dy - dF_dy_num)

# Report results
print(f'Log Likelihood function L w.r.t h')
print(f'dL_dl_num = {dL_dl_num:10.6f}')
print(f'Error:      {err_dL_dl:10.2e}')

print(f'\nObjective function F w.r.t y')
print(f'dF_dy_num = {dF_dy_num:10.6f}')
print(f'Error:      {err_dF_dy:10.2e}')

## Check Second Derivatives Numerically

In [ ]:
# Unpack Hessian of L
d2L_dh2 = hess_L[0, 0]
d2L_dhdl = hess_L[0, 1]
d2L_dldh = hess_L[1, 0]
d2L_dl2 = hess_L[1, 1]

In [ ]:
# d2L_dh2 from symmetric difference
_, grad_h_up, _ = log_like(h=h_up, lam=lam, x=v)
_, grad_h_dn, _ = log_like(h=h_dn, lam=lam, x=v)
d2L_dh2_num = (grad_h_up[0] - grad_h_dn[0]) / dh
d2L_dhdl_num = (grad_h_up[1] - grad_h_dn[1]) / dh

In [ ]:
# d2L_dl2 from symmetric difference
_, grad_l_up, _ = log_like(h=h, lam=lam_up, x=v)
_, grad_l_dn, _ = log_like(h=h, lam=lam_dn, x=v)
d2L_dl2_num = (grad_l_up[1] - grad_l_dn[1]) / dlam
d2L_dldh_num = (grad_l_up[0] - grad_l_dn[0]) / dlam

In [ ]:
# Compare calculated and numerical second derivatives for L
err_d2L_dh2 = np.abs(d2L_dh2 - d2L_dh2_num)
err_d2L_dhdl = np.abs(d2L_dhdl - d2L_dhdl_num)
err_d2L_dldh = np.abs(d2L_dldh - d2L_dldh_num)
err_d2L_dl2 = np.abs(d2L_dl2 - d2L_dl2_num)

In [ ]:
# Unpack Hessian of F
d2F_dx2 = hess_F[0, 0]
d2F_dxdy = hess_F[0, 1]
d2F_dydx = hess_F[1, 0]
d2F_dy2 = hess_F[1, 1]

In [ ]:
# d2F_dx2 from symmetric difference
grad_F_up = jacobian(x_up, v)
grad_F_dn = jacobian(x_dn, v)
d2F_dx2_num = (grad_F_up[0] - grad_F_dn[0]) / dx
d2F_dxdy_num = (grad_F_up[1] - grad_F_dn[1]) / dx

In [ ]:
# d2F_dy2 from symmetric difference
grad_F_up = jacobian(y_up, v)
grad_F_dn = jacobian(y_dn, v)
d2F_dydx_num = (grad_F_up[0] - grad_F_dn[0]) / dy
d2F_dy2_num = (grad_F_up[1] - grad_F_dn[1]) / dy

In [ ]:
# Compare calculated and numerical second derivatives for F
err_d2F_dx2 = np.abs(d2F_dx2 - d2F_dx2_num)
err_d2F_dxdy = np.abs(d2F_dxdy - d2F_dxdy_num)
err_d2F_dydx = np.abs(d2F_dydx - d2F_dydx_num)
err_d2F_dy2 = np.abs(d2F_dy2 - d2F_dy2_num)

In [ ]:
# Report results
print(f'Log Likelihood function L Hessian')
print(f'             Value    ( Error  )')
print(f'd2L_dh2  = {d2L_dh2:10.6f} ({err_d2L_dh2:8.2e})')
print(f'd2L_dhdl = {d2L_dhdl:10.6f} ({err_d2L_dhdl:8.2e})')
print(f'd2L_dldh = {d2L_dldh:10.6f} ({err_d2L_dldh:8.2e})')
print(f'd2L_dl2  = {d2L_dl2:10.6f} ({err_d2L_dl2:8.2e})')

In [ ]:
# Report results
print(f'Objective function F Hessian')
print(f'             Value    ( Error  )')
print(f'd2F_dx2  = {d2F_dx2:10.6f} ({err_d2F_dx2:8.2e})')
print(f'd2F_dxdy = {d2F_dxdy:10.6f} ({err_d2F_dxdy:8.2e})')
print(f'd2F_dydx = {d2F_dydx:10.6f} ({err_d2F_dydx:8.2e})')
print(f'd2F_dy2  = {d2F_dy2:10.6f} ({err_d2F_dy2:8.2e})')

## Symbolic Derivatives of Likelihood

In [ ]:
sympy.init_printing(use_unicode=True)
x, h, lam = sympy.symbols(['x', 'h', r'\lambda'])

In [ ]:
x

In [ ]:
lam

In [ ]:
N = 1 - sympy.exp(-lam)
N

In [ ]:
u = lam * sympy.exp(-lam * x)
u

In [ ]:
g = u / N
g

In [ ]:
dg = diff(g, lam)
dg

In [ ]:
emlx = sympy.exp(-lam*x)
emlx

In [ ]:
du = emlx * (1- lam * x)
du

In [ ]:
dN = 1 - N
dN

In [ ]:
dg_mse = (du*N - u*dN) / N**2
dg_mse

In [ ]:
sympy.simplify(dg_mse - dg)

In [ ]:
substitutions = [(x, 0.5), (lam, 1.0)]

In [ ]:
dg_diff = dg_mse - dg
# dg_diff.subs(substitutions)
sympy.simplify(dg_diff)

In [ ]:
d2u = diff(u, lam, lam)
d2u

In [ ]:
d2u_mse = emlx * (lam * x - 2) * x
d2u_mse

In [ ]:
d2u_mse - d2u

In [ ]:
d2N = diff(N, lam, lam)
d2N

In [ ]:
d2g = diff(g, lam, lam)
d2g

In [ ]:
d2g_mse = (d2u-2*dN*dg)/N + u*dN/N**2

In [ ]:
sympy.simplify(d2g_mse - d2g)

## Fit Mixture Model & Review Results

In [ ]:
# Fit using mixture_fit; wraps up whole calculation
log_like, lam, h = mixture_fit(v)

print(f'Log Likelihood: {log_like:10.3f}')
print(f'h            : {h:10.6f}')
print(f'lambda       :  {lam:10.3e}')

In [ ]:
# Estimated hits
hits_est = v.size * h

# Compare actual and estimated hits
print(f'element_id:     {element_id}')
print(f'Estimated hits: {hits_est}')
print(f'Actual hits:    {hits_actual}')

## Optimize Numericaly with scipy.optimize

In [ ]:
# Set initial guess
h = 0.5
lam = 1.0
x0 = hl2x(h, lam)
h_, lam_ = x0

In [ ]:
# Review initial guess
print(f'Initial guess:')
print(f'h   = {h:9.6f}   (h_   = {h_:9.6f})')
print(f'lam = {lam:9.6f}   (lam_ = {lam_:9.6f})')
print(f'x0 = ', x0)

In [ ]:
# Results table
res_tbl = dict()

In [ ]:
# Minimize with BFGS
method='BFGS'
res = minimize(fun=objective, x0=x0, args=v, method='BFGS', jac=jacobian)

# Save to results table
res_tbl[method] = res.copy()

# Extract h, lam
h, lam = x2hl(res.x)
like = -res.fun
print(f'Optimized with {method}')
print(f'Likelihood: {like:10.3f}')
print(f'h         : {h:10.6f}')
print(f'lambda    :  {lam:10.3e}')

# Display
res

In [ ]:
# Minimize with CG
method='CG'
res = minimize(fun=objective, x0=x0, args=v, method='CG', jac=jacobian)

# Save to results table
res_tbl[method] = res.copy()

# Extract h, lam
h, lam = x2hl(res.x)
like = -res.fun
print(f'Optimized with {method}')
print(f'Likelihood: {like:10.3f}')
print(f'h         : {h:10.6f}')
print(f'lambda    :  {lam:10.3e}')

# Display
res

In [ ]:
# Minimize with Newton-CG (with gradients)
method = 'Newton-CG'
res = minimize(fun=objective, x0=x0, args=v, method='Newton-CG', jac=jacobian, hess=hessian)

# Save to results table
res_tbl[method] = res.copy()

# Extract h, lam
h, lam = x2hl(res.x)
like = -res.fun
print(f'Optimized with {method}')
print(f'Likelihood: {like:10.3f}')
print(f'h         : {h:10.6f}')
print(f'lamba     :  {lam:10.3e}')

# Display
res

## Review Results of Mixture Fit: Hit Count, Resolution

In [ ]:
# Estimated hits
res = res_tbl['Newton-CG']
x0_est = res['x']
h, lam = x2hl(x0_est)
hits_est = v.size * 0.2231

# Compare actual and estimated hits
print(f'element_id:     {element_id}')
print(f'Estimated hits: {hits_est}')
print(f'Actual hits:    {hits_actual}')

In [ ]:
res

In [ ]:
res['x']

In [ ]:
x0